In [47]:
import pandas as pd

In [45]:
pd.set_option('display.max_colwidth',-1)

/Users/mimid/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [3]:
df = pd.read_json('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.json')

In [97]:
df.head()

country  iso_code  \
0  Afghanistan  AFG        
1  Africa       OWID_AFR   
2  Albania      ALB        
3  Algeria      DZA        
4  Andorra      AND        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [98]:
us = df.loc[df.iso_code == 'USA', 'data'].values[0]

us_df = pd.DataFrame(us)
us_df.to_csv('us.csv')
us_df.head()

,date,total_vaccinations,people_vaccinated,total_vaccinations_per_hundred,people_vaccinated_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,people_fully_vaccinated,people_fully_vaccinated_per_hundred
0,2020-12-20,556208.0,556208.0,0.17,0.17,NaN,NaN,NaN,NaN,NaN
1,2020-12-21,614117.0,614117.0,0.18,0.18,57909.0,57909.0,173.0,NaN,NaN
2,2020-12-22,NaN,NaN,NaN,NaN,NaN,127432.0,381.0,NaN,NaN
3,2020-12-23,1008025.0,1008025.0,0.30,0.30,NaN,150606.0,450.0,NaN,NaN
4,2020-12-24,NaN,NaN,NaN,NaN,NaN,191001.0,571.0,NaN,NaN


In [99]:
isr = df.loc[df.iso_code == 'ISR', 'data'].values[0]

isr_df = pd.DataFrame(isr)
isr_df.to_csv('israel.csv')
isr_df.head()

,date,total_vaccinations,people_vaccinated,total_vaccinations_per_hundred,people_vaccinated_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,people_fully_vaccinated,people_fully_vaccinated_per_hundred
0,2020-12-19,61,61,0.00,0.00,NaN,NaN,NaN,NaN,NaN
1,2020-12-20,7432,7432,0.09,0.09,7371.0,7371.0,852.0,NaN,NaN
2,2020-12-21,32302,32302,0.37,0.37,24870.0,16120.0,1862.0,NaN,NaN
3,2020-12-22,76910,76910,0.89,0.89,44608.0,25616.0,2959.0,NaN,NaN
4,2020-12-23,139717,139717,1.61,1.61,62807.0,34914.0,4034.0,NaN,NaN
